## Start DB with podman

In [1]:
!{ podman stop postgres && podman rm postgres; } || true
!podman run --rm --name postgres -e POSTGRES_PASSWORD=hwpass -e POSTGRES_DB=homework -e POSTGRES_USER=homework -p 5432:5432 -d postgres:16

postgres
Error: no container with ID or name "postgres" found: no such container
c49b01074eeb3407b08b2f36e33f616fc737629b7bdc9232e9652c1953318145


In [2]:
!podman ps -a

CONTAINER ID  IMAGE                          COMMAND     CREATED                 STATUS                 PORTS                   NAMES
c49b01074eeb  docker.io/library/postgres:16  postgres    Less than a second ago  Up Less than a second  0.0.0.0:5432->5432/tcp  postgres


In [3]:
!podman logs postgres

The files belonging to this database system will be owned by user "postgres".
This user must also own the server process.

The database cluster will be initialized with locale "en_US.utf8".
The default database encoding has accordingly been set to "UTF8".
The default text search configuration will be set to "english".

Data page checksums are disabled.

fixing permissions on existing directory /var/lib/postgresql/data ... ok
creating subdirectories ... ok
selecting dynamic shared memory implementation ... posix
selecting default max_connections ... 100
selecting default shared_buffers ... 128MB
selecting default time zone ... Etc/UTC
creating configuration files ... ok
running bootstrap script ... ok
performing post-bootstrap initialization ... ok
syncing data to disk ... ok


Success. You can now start the database server using:

    pg_ctl -D /var/lib/postgresql/data -l logfile start

initdb: warning: enabling "trust" authentication for local connections
initdb: hint: You can change 

## DB structure

![image](database.png)
Таблицы приводятся к 2НФ (раньше в таблице transaction было 1НФ, но были зависимости от product_id)
К 3НФ привести не получилось, так как не совсем понимаю, как реализовать one-to-one взаимоотношения в таблице customer между адресом и cutomer_id

## Preparing connection

In [4]:
import psycopg2 as pg
import pandas as pd
import numpy as np

from typing import Any

/var/folders/4p/7625v4fd3vnfkh0pgq24l678krwr8q/T/ipykernel_11471/640772971.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:
conn = pg.connect(dbname="homework", user="homework", password="hwpass", host="localhost")
cursor = conn.cursor()

## Creating tables

In [6]:
with open("database.sql") as f:
    queries = f.read()
print(queries)

CREATE TABLE "brand" (
  "id" serial PRIMARY KEY NOT NULL,
  "brand" varchar NOT NULL
);

CREATE TABLE "product_line" (
  "id" serial PRIMARY KEY NOT NULL,
  "product_line" varchar NOT NULL
);

CREATE TABLE "product_class" (
  "id" serial PRIMARY KEY NOT NULL,
  "product_class" varchar NOT NULL
);

CREATE TABLE "product_size" (
  "id" serial PRIMARY KEY NOT NULL,
  "product_size" varchar NOT NULL
);

CREATE TABLE "product" (
  "id" serial PRIMARY KEY NOT NULL,
  "product_id" integer NOT NULL,
  "brand_id" integer,
  "product_line_id" integer,
  "product_class_id" integer,
  "product_size_id" integer,
  "standard_cost" numeric(6,2)
);

CREATE TABLE "gender" (
  "id" serial PRIMARY KEY NOT NULL,
  "gender" character(1) NOT NULL
);

CREATE TABLE "job_industry_category" (
  "id" serial PRIMARY KEY NOT NULL,
  "job_industry_category" varchar NOT NULL
);

CREATE TABLE "wealth_segment" (
  "id" serial PRIMARY KEY NOT NULL,
  "wealth_segment" varchar NOT NULL
);

CREATE TABLE "country" (
  "id

In [7]:
cursor.execute(queries)
conn.commit()

## Reading data

In [8]:
xl = pd.ExcelFile('data.xlsx')
xl.sheet_names

['transaction', 'customer']

In [9]:
transaction = xl.parse("transaction")
transaction.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost
0,1,2,2950,2017-02-25,False,Approved,Solex,Standard,medium,medium,71.49,53.62
1,2,3,3120,2017-05-21,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92
2,3,37,402,2017-10-16,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82
3,4,88,3135,2017-08-31,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10
4,5,78,787,2017-10-01,True,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48


In [10]:
customer = xl.parse("customer")
customer.head()

,customer_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,address,postcode,state,country,property_valuation
0,1,Laraine,Medendorp,F,1953-10-12 00:00:00,Executive Secretary,Health,Mass Customer,N,Yes,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,Eli,Bockman,Male,1980-12-16 00:00:00,Administrative Officer,Financial Services,Mass Customer,N,Yes,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,3,Arlin,Dearle,Male,1954-01-20 00:00:00,Recruiting Manager,Property,Mass Customer,N,Yes,0 Holy Cross Court,4211,QLD,Australia,9
3,4,Talbot,NaN,Male,1961-10-03 00:00:00,NaN,IT,Mass Customer,N,No,17979 Del Mar Point,2448,New South Wales,Australia,4
4,5,Sheila-kathryn,Calton,Female,1977-05-13 00:00:00,Senior Editor,NaN,Affluent Customer,N,Yes,9 Oakridge Court,3216,VIC,Australia,9


In [11]:
customer.gender = customer.gender.map(lambda x: {"Female": "F", "Male": "M", "Femal": "F"}.get(x, x))

## Insert data into DB

In [12]:
def join_row(arr: list) -> str:
    result = ""
    for v in arr:
        if pd.isnull(v):
            result += "NULL,"
        elif isinstance(v, int | float):
            result += str(v) + ","
        else:
            result += "'" + str(v) + "',"
    return result.rstrip(",")
    
def join_values(arr: list) -> str:
    return "(" + "), (".join(arr) + ")"

def join_cols(arr: list) -> str:
    return '("' + '", "'.join(arr) + '")'

In [13]:
def tuple_list_to_dict(values: tuple[int, str]) -> dict[str, int]:
    return {(tup[1] if len(tup) <= 2 else tup[1:]): tup[0] for tup in values}

In [14]:
def insert_into(table: str, cols: list[str], values: pd.DataFrame) -> list:
    insert_query = f"INSERT INTO {table} {join_cols(cols)} VALUES {join_values([join_row(val) for val in values.drop_duplicates().to_numpy()])};"
    print(insert_query)
    
    cursor.execute(insert_query)
    conn.commit()
    cursor.execute(f"select * from {table}")
    conn.commit()
    return cursor.fetchall()

In [15]:
brand = tuple_list_to_dict(insert_into("brand", ["brand"], transaction[["brand"]].dropna()))
product_line = tuple_list_to_dict(insert_into("product_line", ["product_line"], transaction[["product_line"]].dropna()))
product_class = tuple_list_to_dict(insert_into("product_class", ["product_class"], transaction[["product_class"]].dropna()))
product_size = tuple_list_to_dict(insert_into("product_size", ["product_size"], transaction[["product_size"]].dropna()))

order_status = tuple_list_to_dict(insert_into("order_status", ["order_status"], transaction[["order_status"]].dropna()))

gender = tuple_list_to_dict(insert_into("gender", ["gender"], customer[["gender"]].dropna()))
job_industry_category = tuple_list_to_dict(insert_into("job_industry_category", ["job_industry_category"], customer[["job_industry_category"]].dropna()))
wealth_segment = tuple_list_to_dict(insert_into("wealth_segment", ["wealth_segment"], customer[["wealth_segment"]].dropna()))
country = tuple_list_to_dict(insert_into("country", ["country"], customer[["country"]].dropna()))

INSERT INTO brand ("brand") VALUES ('Solex'), ('Trek Bicycles'), ('OHM Cycles'), ('Norco Bicycles'), ('Giant Bicycles'), ('WeareA2B');
INSERT INTO product_line ("product_line") VALUES ('Standard'), ('Road'), ('Mountain'), ('Touring');
INSERT INTO product_class ("product_class") VALUES ('medium'), ('low'), ('high');
INSERT INTO product_size ("product_size") VALUES ('medium'), ('large'), ('small');
INSERT INTO order_status ("order_status") VALUES ('Approved'), ('Cancelled');
INSERT INTO gender ("gender") VALUES ('F'), ('M'), ('U');
INSERT INTO job_industry_category ("job_industry_category") VALUES ('Health'), ('Financial Services'), ('Property'), ('IT'), ('Retail'), ('Argiculture'), ('Manufacturing'), ('Telecommunications'), ('Entertainment');
INSERT INTO wealth_segment ("wealth_segment") VALUES ('Mass Customer'), ('Affluent Customer'), ('High Net Worth');
INSERT INTO country ("country") VALUES ('Australia');


In [16]:
print(brand)
print(product_line)
print(product_class)
print(product_size)

print(order_status)


print(gender)
print(job_industry_category)
print(wealth_segment)
print(country)

{'Solex': 1, 'Trek Bicycles': 2, 'OHM Cycles': 3, 'Norco Bicycles': 4, 'Giant Bicycles': 5, 'WeareA2B': 6}
{'Standard': 1, 'Road': 2, 'Mountain': 3, 'Touring': 4}
{'medium': 1, 'low': 2, 'high': 3}
{'medium': 1, 'large': 2, 'small': 3}
{'Approved': 1, 'Cancelled': 2}
{'F': 1, 'M': 2, 'U': 3}
{'Health': 1, 'Financial Services': 2, 'Property': 3, 'IT': 4, 'Retail': 5, 'Argiculture': 6, 'Manufacturing': 7, 'Telecommunications': 8, 'Entertainment': 9}
{'Mass Customer': 1, 'Affluent Customer': 2, 'High Net Worth': 3}
{'Australia': 1}


In [17]:
customer.gender = customer.gender.map(gender)
customer.job_industry_category = customer.job_industry_category.map(job_industry_category.get)
customer.wealth_segment = customer.wealth_segment.map(wealth_segment)
customer.country = customer.country.map(country)
customer.head()

,customer_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,address,postcode,state,country,property_valuation
0,1,Laraine,Medendorp,1,1953-10-12 00:00:00,Executive Secretary,1.0,1,N,Yes,060 Morning Avenue,2016,New South Wales,1,10
1,2,Eli,Bockman,2,1980-12-16 00:00:00,Administrative Officer,2.0,1,N,Yes,6 Meadow Vale Court,2153,New South Wales,1,10
2,3,Arlin,Dearle,2,1954-01-20 00:00:00,Recruiting Manager,3.0,1,N,Yes,0 Holy Cross Court,4211,QLD,1,9
3,4,Talbot,NaN,2,1961-10-03 00:00:00,NaN,4.0,1,N,No,17979 Del Mar Point,2448,New South Wales,1,4
4,5,Sheila-kathryn,Calton,1,1977-05-13 00:00:00,Senior Editor,NaN,2,N,Yes,9 Oakridge Court,3216,VIC,1,9


In [18]:
state = tuple_list_to_dict(insert_into("state", ["state", "country_id"], customer[["state", "country"]]))

INSERT INTO state ("state", "country_id") VALUES ('New South Wales',1), ('QLD',1), ('VIC',1), ('NSW',1), ('Victoria',1);


In [19]:
print(state)

{('New South Wales', 1): 1, ('QLD', 1): 2, ('VIC', 1): 3, ('NSW', 1): 4, ('Victoria', 1): 5}


In [20]:
customer.state = customer.apply(lambda row: state[(row.state, row.country)], axis=1)

In [21]:
customer.first_name = customer.first_name.map(lambda x: x.replace("'", "''"))
customer.last_name = customer.last_name.map(lambda x: x.replace("'", "''") if isinstance(x, str) else x)

In [22]:
customer.head()

,customer_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,address,postcode,state,country,property_valuation
0,1,Laraine,Medendorp,1,1953-10-12 00:00:00,Executive Secretary,1.0,1,N,Yes,060 Morning Avenue,2016,1,1,10
1,2,Eli,Bockman,2,1980-12-16 00:00:00,Administrative Officer,2.0,1,N,Yes,6 Meadow Vale Court,2153,1,1,10
2,3,Arlin,Dearle,2,1954-01-20 00:00:00,Recruiting Manager,3.0,1,N,Yes,0 Holy Cross Court,4211,2,1,9
3,4,Talbot,NaN,2,1961-10-03 00:00:00,NaN,4.0,1,N,No,17979 Del Mar Point,2448,1,1,4
4,5,Sheila-kathryn,Calton,1,1977-05-13 00:00:00,Senior Editor,NaN,2,N,Yes,9 Oakridge Court,3216,3,1,9


In [23]:
insert_into(
    "customer", 
    ["id", "first_name", "last_name", "gender_id", "DOB", "job_title", "job_industry_category_id", "wealth_segment_id", "deceased_indicator", "owns_car", "address", "postcode", "state_id", "property_valuation"],
    customer[["customer_id", "first_name", "last_name", "gender", "DOB", "job_title", "job_industry_category", "wealth_segment", "deceased_indicator", "owns_car", "address", "postcode", "state", "property_valuation"]],
)

INSERT INTO customer ("id", "first_name", "last_name", "gender_id", "DOB", "job_title", "job_industry_category_id", "wealth_segment_id", "deceased_indicator", "owns_car", "address", "postcode", "state_id", "property_valuation") VALUES (1,'Laraine','Medendorp',1,'1953-10-12 00:00:00','Executive Secretary',1.0,1,'N','Yes','060 Morning Avenue',2016,1,10), (2,'Eli','Bockman',2,'1980-12-16 00:00:00','Administrative Officer',2.0,1,'N','Yes','6 Meadow Vale Court',2153,1,10), (3,'Arlin','Dearle',2,'1954-01-20 00:00:00','Recruiting Manager',3.0,1,'N','Yes','0 Holy Cross Court',4211,2,9), (4,'Talbot',NULL,2,'1961-10-03 00:00:00',NULL,4.0,1,'N','No','17979 Del Mar Point',2448,1,4), (5,'Sheila-kathryn','Calton',1,'1977-05-13 00:00:00','Senior Editor',NULL,2,'N','Yes','9 Oakridge Court',3216,3,9), (6,'Curr','Duckhouse',2,'1966-09-16 00:00:00',NULL,5.0,3,'N','Yes','4 Delaware Trail',2210,1,9), (7,'Fina','Merali',1,'1976-02-23 00:00:00',NULL,2.0,2,'N','Yes','49 Londonderry Lane',2650,1,4), (8,'Rod','

[(1,
  'Laraine',
  'Medendorp',
  1,
  datetime.date(1953, 10, 12),
  'Executive Secretary',
  1,
  1,
  False,
  True,
  '060 Morning Avenue',
  2016,
  1,
  10),
 (2,
  'Eli',
  'Bockman',
  2,
  datetime.date(1980, 12, 16),
  'Administrative Officer',
  2,
  1,
  False,
  True,
  '6 Meadow Vale Court',
  2153,
  1,
  10),
 (3,
  'Arlin',
  'Dearle',
  2,
  datetime.date(1954, 1, 20),
  'Recruiting Manager',
  3,
  1,
  False,
  True,
  '0 Holy Cross Court',
  4211,
  2,
  9),
 (4,
  'Talbot',
  None,
  2,
  datetime.date(1961, 10, 3),
  None,
  4,
  1,
  False,
  False,
  '17979 Del Mar Point',
  2448,
  1,
  4),
 (5,
  'Sheila-kathryn',
  'Calton',
  1,
  datetime.date(1977, 5, 13),
  'Senior Editor',
  None,
  2,
  False,
  True,
  '9 Oakridge Court',
  3216,
  3,
  9),
 (6,
  'Curr',
  'Duckhouse',
  2,
  datetime.date(1966, 9, 16),
  None,
  5,
  3,
  False,
  True,
  '4 Delaware Trail',
  2210,
  1,
  9),
 (7,
  'Fina',
  'Merali',
  1,
  datetime.date(1976, 2, 23),
  None,
  

In [24]:
transaction.brand = transaction.brand.map(brand)
transaction.product_line = transaction.product_line.map(product_line)
transaction.product_class = transaction.product_class.map(product_class)
transaction.product_size = transaction.product_size.map(product_size)
transaction

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost
0,1,2,2950,2017-02-25,False,Approved,1.0,1.0,1.0,1.0,71.49,53.62
1,2,3,3120,2017-05-21,True,Approved,2.0,1.0,1.0,2.0,2091.47,388.92
2,3,37,402,2017-10-16,False,Approved,3.0,1.0,2.0,1.0,1793.43,248.82
3,4,88,3135,2017-08-31,False,Approved,4.0,1.0,1.0,1.0,1198.46,381.10
4,5,78,787,2017-10-01,True,Approved,5.0,1.0,1.0,2.0,1765.30,709.48
...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19996,51,1018,2017-06-24,True,Approved,3.0,1.0,3.0,1.0,2005.66,1203.40
19996,19997,41,127,2017-11-09,True,Approved,1.0,2.0,1.0,1.0,416.98,312.74
19997,19998,87,2284,2017-04-14,True,Approved,3.0,1.0,1.0,1.0,1636.90,44.71
19998,19999,6,2764,2017-07-03,False,Approved,3.0,1.0,3.0,1.0,227.88,136.73


In [25]:
product_id = tuple_list_to_dict(insert_into(
    "product", 
    ["product_id", "brand_id", "product_line_id", "product_class_id", "product_size_id", "standard_cost"],
    transaction[["product_id", "brand", "product_line", "product_class", "product_size", "standard_cost"]],
))

INSERT INTO product ("product_id", "brand_id", "product_line_id", "product_class_id", "product_size_id", "standard_cost") VALUES (2.0,1.0,1.0,1.0,1.0,53.62), (3.0,2.0,1.0,1.0,2.0,388.92), (37.0,3.0,1.0,2.0,1.0,248.82), (88.0,4.0,1.0,1.0,1.0,381.1), (78.0,5.0,1.0,1.0,2.0,709.48), (25.0,5.0,2.0,1.0,1.0,829.65), (22.0,6.0,1.0,1.0,1.0,45.26), (15.0,6.0,1.0,1.0,1.0,13.44), (67.0,1.0,1.0,1.0,2.0,380.74), (12.0,6.0,1.0,1.0,1.0,161.6), (5.0,2.0,3.0,2.0,1.0,459.71), (61.0,3.0,1.0,2.0,1.0,56.93), (35.0,2.0,1.0,2.0,1.0,154.4), (16.0,4.0,1.0,3.0,3.0,1479.11), (12.0,5.0,1.0,1.0,2.0,709.48), (79.0,4.0,1.0,1.0,1.0,818.01), (33.0,5.0,1.0,1.0,3.0,1167.18), (54.0,6.0,1.0,1.0,1.0,13.44), (27.0,2.0,1.0,1.0,1.0,388.72), (82.0,5.0,2.0,1.0,1.0,829.65), (89.0,6.0,4.0,1.0,2.0,57.74), (64.0,2.0,1.0,1.0,2.0,596.55), (19.0,2.0,3.0,2.0,1.0,459.71), (72.0,4.0,1.0,1.0,1.0,270.3), (91.0,6.0,1.0,2.0,1.0,513.85), (1.0,5.0,1.0,1.0,1.0,954.82), (99.0,2.0,2.0,2.0,3.0,1531.42), (0.0,4.0,2.0,1.0,1.0,376.84), (92.0,6.0,1.0,1

In [26]:
from decimal import Decimal

In [27]:
transaction.order_status = transaction.order_status.map(order_status)
transaction.standard_cost = transaction.standard_cost.map(lambda x: Decimal(str(np.round(x, 2))))
transaction.product_id = transaction.apply(lambda row: product_id[tuple(row[["product_id", "brand", "product_line", "product_class", "product_size", "standard_cost"]])] if not pd.isnull(row.brand) else product_id[(0, None, None, None, None, None)], axis=1)

In [28]:
transaction[transaction.customer_id > 4000]

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost
8707,8708,202,5034,2017-10-07,False,1,1.0,2.0,1.0,1.0,416.98,312.74
16700,16701,203,5034,2017-01-27,False,1,4.0,1.0,1.0,1.0,360.40,270.3
17468,17469,204,5034,2017-01-03,False,1,3.0,2.0,1.0,1.0,742.54,667.4


In [29]:
# Убираем транзакции с пользователями, которых не существует в базе
transaction = transaction[transaction.customer_id <= 4000]
transaction

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost
0,1,1,2950,2017-02-25,False,1,1.0,1.0,1.0,1.0,71.49,53.62
1,2,2,3120,2017-05-21,True,1,2.0,1.0,1.0,2.0,2091.47,388.92
2,3,3,402,2017-10-16,False,1,3.0,1.0,2.0,1.0,1793.43,248.82
3,4,4,3135,2017-08-31,False,1,4.0,1.0,1.0,1.0,1198.46,381.1
4,5,5,787,2017-10-01,True,1,5.0,1.0,1.0,2.0,1765.30,709.48
...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19996,123,1018,2017-06-24,True,1,3.0,1.0,3.0,1.0,2005.66,1203.4
19996,19997,99,127,2017-11-09,True,1,1.0,2.0,1.0,1.0,416.98,312.74
19997,19998,147,2284,2017-04-14,True,1,3.0,1.0,1.0,1.0,1636.90,44.71
19998,19999,42,2764,2017-07-03,False,1,3.0,1.0,3.0,1.0,227.88,136.73


In [30]:
insert_into(
    "transaction", 
    ["product_id", "customer_id", "transaction_date", "online_order", "order_status_id", "list_price"],
    transaction[["product_id", "customer_id", "transaction_date", "online_order", "order_status", "list_price"]],
)

INSERT INTO transaction ("product_id", "customer_id", "transaction_date", "online_order", "order_status_id", "list_price") VALUES (1,2950,'2017-02-25 00:00:00',False,1,71.49), (2,3120,'2017-05-21 00:00:00',True,1,2091.47), (3,402,'2017-10-16 00:00:00',False,1,1793.43), (4,3135,'2017-08-31 00:00:00',False,1,1198.46), (5,787,'2017-10-01 00:00:00',True,1,1765.3), (6,2339,'2017-03-08 00:00:00',True,1,1538.99), (7,1542,'2017-04-21 00:00:00',True,1,60.34), (8,2459,'2017-07-15 00:00:00',False,1,1292.84), (9,1305,'2017-08-10 00:00:00',False,1,1071.23), (10,3262,'2017-08-30 00:00:00',True,1,1231.15), (11,1986,'2017-01-17 00:00:00',False,1,574.64), (12,2783,'2017-01-05 00:00:00',True,1,71.16), (13,1243,'2017-02-26 00:00:00',True,1,1057.51), (14,2717,'2017-09-10 00:00:00',False,1,1661.92), (15,247,'2017-06-11 00:00:00',False,1,1765.3), (2,2961,'2017-10-10 00:00:00',False,1,2091.47), (16,2426,'2017-04-03 00:00:00',False,1,1555.58), (17,1842,'2017-06-02 00:00:00',False,1,1311.44), (18,2268,'2017-04

[(1, 1, 2950, datetime.date(2017, 2, 25), False, 1, Decimal('71.49')),
 (2, 2, 3120, datetime.date(2017, 5, 21), True, 1, Decimal('2091.47')),
 (3, 3, 402, datetime.date(2017, 10, 16), False, 1, Decimal('1793.43')),
 (4, 4, 3135, datetime.date(2017, 8, 31), False, 1, Decimal('1198.46')),
 (5, 5, 787, datetime.date(2017, 10, 1), True, 1, Decimal('1765.30')),
 (6, 6, 2339, datetime.date(2017, 3, 8), True, 1, Decimal('1538.99')),
 (7, 7, 1542, datetime.date(2017, 4, 21), True, 1, Decimal('60.34')),
 (8, 8, 2459, datetime.date(2017, 7, 15), False, 1, Decimal('1292.84')),
 (9, 9, 1305, datetime.date(2017, 8, 10), False, 1, Decimal('1071.23')),
 (10, 10, 3262, datetime.date(2017, 8, 30), True, 1, Decimal('1231.15')),
 (11, 11, 1986, datetime.date(2017, 1, 17), False, 1, Decimal('574.64')),
 (12, 12, 2783, datetime.date(2017, 1, 5), True, 1, Decimal('71.16')),
 (13, 13, 1243, datetime.date(2017, 2, 26), True, 1, Decimal('1057.51')),
 (14, 14, 2717, datetime.date(2017, 9, 10), False, 1, Decima

In [31]:
cursor.close()
conn.close()